In [ ]:
# %% [markdown]
# # Laboratorium Evaluasi Sistem Rekomendasi Adaptif
#
# **Nama:** M Egypt Pratama
#
# **Tujuan Notebook:**
# Notebook ini berfungsi sebagai lingkungan eksperimental untuk mengevaluasi kinerja sistem rekomendasi adaptif (MAB-MMR) yang diusulkan dalam tesis. Tujuannya adalah untuk membandingkan model yang diusulkan dengan dua model dasar (*baseline*) berdasarkan serangkaian metrik kuantitatif yang telah ditentukan, sesuai dengan protokol evaluasi di Bab III tesis.

# %%

In [1]:
# =============================================================================
# BAGIAN 1: PENGATURAN & PERSIAPAN LINGKUNGAN
# =============================================================================
print("BAGIAN 1: PENGATURAN & PERSIAPAN LINGKUNGAN")

# Sel 1: Import Pustaka Utama
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import sys
import os
import warnings

warnings.filterwarnings('ignore')

# Mengatur gaya plot agar terlihat profesional
sns.set_theme(style="whitegrid")
print("✅ Pustaka utama berhasil di-import.")

# Sel 2: Hubungkan ke Kode Proyek Aplikasi
# Penting: Pastikan folder 'backend' Anda berada di direktori yang sama dengan notebook ini,
# atau sesuaikan path-nya.
module_path = os.path.abspath(os.path.join('.', 'backend'))
if module_path not in sys.path:
    sys.path.append(module_path)

try:
    from app.core.db import get_db
    from app.services.ml_service import MLService
    print("✅ Berhasil terhubung dengan modul aplikasi.")
except ImportError as e:
    print(f"❌ Gagal terhubung dengan modul aplikasi. Pastikan folder 'backend' ada di path yang benar.")
    print(f"Error: {e}")

# Sel 3: Inisialisasi Koneksi Database dan Layanan Utama
try:
    db_session_generator = get_db()
    db = next(db_session_generator)
    print("✅ Koneksi database berhasil dibuat.")

    print("⏳ Menginisialisasi MLService (ini mungkin butuh beberapa saat)...")
    ml_service = MLService(db)
    print("✅ MLService siap digunakan.")
except Exception as e:
    print(f"❌ Gagal menginisialisasi layanan. Pastikan database berjalan dan konfigurasi benar.")
    print(f"Error: {e}")


BAGIAN 1: PENGATURAN & PERSIAPAN LINGKUNGAN
✅ Pustaka utama berhasil di-import.
❌ Gagal terhubung dengan modul aplikasi. Pastikan folder 'backend' ada di path yang benar.
Error: No module named 'app'
❌ Gagal menginisialisasi layanan. Pastikan database berjalan dan konfigurasi benar.
Error: name 'get_db' is not defined

BAGIAN 2: PEMUATAN DAN PEMBAGIAN DATA


NameError: name 'ml_service' is not defined

In [ ]:
# =============================================================================
# BAGIAN 2: PEMUATAN DAN PEMBAGIAN DATA
# =============================================================================
print("\nBAGIAN 2: PEMUATAN DAN PEMBAGIAN DATA")
from sklearn.model_selection import train_test_split

# Memuat data rating lengkap dari MLService yang sudah diinisialisasi
all_ratings_df = ml_service.collaborative_recommender.ratings_df
all_destinations_df = ml_service.content_recommender.destinations_df

# Membagi data menjadi train/test
train_df, test_df = train_test_split(all_ratings_df, test_size=0.2, random_state=42, stratify=all_ratings_df['user_id'])

print(f"Jumlah total ratings: {len(all_ratings_df)}")
print(f"Ukuran data latih (train): {len(train_df)}")
print(f"Ukuran data uji (test): {len(test_df)}")

# Menyimpan user dan item unik untuk evaluasi
test_users = test_df['user_id'].unique()
all_item_ids = all_destinations_df['id'].unique()
print(f"Jumlah pengguna unik di data uji: {len(test_users)}")
print(f"Jumlah total destinasi unik: {len(all_item_ids)}")

# %% [markdown]
# ---
# ## BAGIAN 3: IMPLEMENTASI MODEL PEMBANDING (*BASELINE*)
#
# Kita mendefinisikan dan melatih model pembanding yang akan digunakan sebagai tolak ukur:
# 1.  **Popularity Recommender**: Merekomendasikan item paling populer.
# 2.  **Standard Collaborative Filtering**: Model CF tanpa re-ranking MMR.

In [ ]:
# =============================================================================
# BAGIAN 3: IMPLEMENTASI MODEL PEMBANDING (BASELINE)
# =============================================================================
print("\nBAGIAN 3: IMPLEMENTASI MODEL PEMBANDING (BASELINE)")

class PopularityRecommender:
    def __init__(self):
        self.top_n = None
        self.name = "Popularity"

    def fit(self, train_data):
        print("Melatih Model Popularitas...")
        popularity_scores = train_data.groupby('destination_id').size().sort_values(ascending=False)
        self.top_n = popularity_scores.index.tolist()
        print("✅ Model Popularitas siap.")

    def predict(self, user_id, num_recommendations=10):
        return self.top_n[:num_recommendations]

class StandardCFRecommender:
    def __init__(self, ml_service_instance):
        self.recommender = ml_service_instance.collaborative_recommender
        self.name = "Collaborative Filtering"

    def fit(self, train_data):
        # Model CF di MLService sudah dilatih dengan data penuh.
        # Untuk eksperimen yang adil, idealnya kita melatih ulang hanya dengan train_data.
        # Untuk saat ini, kita akan gunakan model yang sudah ada sebagai proxy.
        print("✅ Model Collaborative Filtering (dari MLService) siap.")

    def predict(self, user_id, num_recommendations=10):
        recs_df = self.recommender.predict(user_id, num_recommendations)
        return recs_df['destination_id'].tolist()

# Inisialisasi dan latih model baseline
pop_recommender = PopularityRecommender()
pop_recommender.fit(train_df)

cf_recommender = StandardCFRecommender(ml_service)
cf_recommender.fit(train_df)

# Untuk model adaptif kita, kita akan panggil langsung dari ml_service
adaptive_recommender = ml_service

In [2]:
# =============================================================================
# BAGIAN 4: FUNGSI METRIK EVALUASI
# =============================================================================
print("\nBAGIAN 4: FUNGSI METRIK EVALUASI")

def precision_at_k(recommended_items, actual_items, k):
    if not recommended_items: return 0.0
    actual_set = set(actual_items)
    recommended_k = recommended_items[:k]
    hits = len(set(recommended_k).intersection(actual_set))
    return hits / k

def recall_at_k(recommended_items, actual_items, k):
    if not recommended_items or not actual_items: return 0.0
    actual_set = set(actual_items)
    recommended_k = recommended_items[:k]
    hits = len(set(recommended_k).intersection(actual_set))
    return hits / len(actual_set)

def ndcg_at_k(recommended_items, actual_items, k):
    if not recommended_items: return 0.0
    actual_set = set(actual_items)
    dcg = 0.0
    for i, item in enumerate(recommended_items[:k]):
        if item in actual_set:
            dcg += 1.0 / np.log2(i + 2)

    idcg = sum(1.0 / np.log2(i + 2) for i in range(min(len(actual_set), k)))
    return dcg / idcg if idcg > 0 else 0.0

def intra_list_diversity(recommended_items, similarity_matrix):
    if len(recommended_items) < 2: return 0.0
    total_dissimilarity = 0
    pair_count = 0
    for i in range(len(recommended_items)):
        for j in range(i + 1, len(recommended_items)):
            id1, id2 = recommended_items[i], recommended_items[j]
            try:
                similarity = similarity_matrix.loc[id1, id2]
                total_dissimilarity += (1 - similarity)
                pair_count += 1
            except KeyError:
                continue
    return total_dissimilarity / pair_count if pair_count > 0 else 0.0

def catalog_coverage(all_recommendations_lists, all_item_ids):
    recommended_items_set = set()
    for rec_list in all_recommendations_lists:
        recommended_items_set.update(rec_list)
    return len(recommended_items_set) / len(all_item_ids)

def novelty(recommended_items, popularity_scores):
    if not recommended_items: return 0.0
    total_novelty = 0
    max_pop = popularity_scores.max()
    for item in recommended_items:
        # Self-information: -log2(p(i))
        # p(i) adalah probabilitas item, diaproksimasi dengan skor popularitas ternormalisasi
        item_pop = popularity_scores.get(item, 1) # Default 1 jika tidak ada
        prob = item_pop / max_pop if max_pop > 0 else 0
        total_novelty += -np.log2(prob + 1e-9) # Tambah epsilon untuk hindari log(0)
    return total_novelty / len(recommended_items)

print("✅ Semua fungsi metrik telah didefinisikan.")


In [ ]:
# =============================================================================
# BAGIAN 5: PELAKSANAAN EKSPERIMEN
# =============================================================================
print("\nBAGIAN 5: PELAKSANAAN EKSPERIMEN")
k = 10
results = {}
models = {
    "Adaptive (MAB-MMR)": adaptive_recommender,
    "Collaborative Filtering": cf_recommender,
    "Popularity": pop_recommender
}

# Siapkan data ground truth: destinasi yang benar-benar disukai pengguna di data uji
ground_truth = test_df[test_df['rating'] >= 4].groupby('user_id')['destination_id'].apply(list).to_dict()
similarity_matrix = ml_service.hybrid_recommender.similarity_matrix
popularity_scores = train_df['destination_id'].value_counts()


for model_name, model in models.items():
    print(f"⏳ Mengevaluasi model: {model_name}...")
    all_recs_for_model = []
    metrics = {
        'precision': [], 'recall': [], 'ndcg': [],
        'diversity': [], 'novelty': []
    }

    # Kita hanya uji pada sebagian kecil pengguna agar tidak terlalu lama
    users_to_test = test_users[:50]

    for user_id in tqdm(users_to_test, desc=f"Pengguna ({model_name})"):
        if model_name == "Adaptive (MAB-MMR)":
            recs_df, _, _ = model.get_recommendations(user_id, num_recommendations=k)
            recommended_ids = recs_df['destination_id'].tolist()
        else:
            recommended_ids = model.predict(user_id, num_recommendations=k)

        all_recs_for_model.append(recommended_ids)
        actual_items = ground_truth.get(user_id, [])

        # Hitung metrik per pengguna
        metrics['precision'].append(precision_at_k(recommended_ids, actual_items, k))
        metrics['recall'].append(recall_at_k(recommended_ids, actual_items, k))
        metrics['ndcg'].append(ndcg_at_k(recommended_ids, actual_items, k))
        metrics['diversity'].append(intra_list_diversity(recommended_ids, similarity_matrix))
        metrics['novelty'].append(novelty(recommended_ids, popularity_scores))

    # Hitung rata-rata metrik dan coverage
    avg_metrics = {key: np.mean(val) for key, val in metrics.items()}
    avg_metrics['coverage'] = catalog_coverage(all_recs_for_model, all_item_ids)
    results[model_name] = avg_metrics

print("\n✅ Eksperimen selesai.")

In [2]:
# =============================================================================
# BAGIAN 6: ANALISIS DAN VISUALISASI HASIL
# =============================================================================
print("\nBAGIAN 6: ANALISIS DAN VISUALISASI HASIL")

# Buat DataFrame dari hasil
results_df = pd.DataFrame(results).T.rename(columns={
    'precision': f'Precision@{k}',
    'recall': f'Recall@{k}',
    'ndcg': f'NDCG@{k}',
    'diversity': 'Diversity',
    'novelty': 'Novelty',
    'coverage': 'Coverage'
})

print("\n--- Tabel Hasil Akhir Metrik Kuantitatif ---")
display(results_df)

# Visualisasi Hasil
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Perbandingan Kinerja Model Rekomendasi', fontsize=20, y=1.02)
axes = axes.flatten()

for i, col in enumerate(results_df.columns):
    sns.barplot(x=results_df.index, y=results_df[col], ax=axes[i])
    axes[i].set_title(col, fontsize=14)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("Skor")
    axes[i].tick_params(axis='x', rotation=15)
    # Tambahkan label angka di atas bar
    for p in axes[i].patches:
        axes[i].annotate(f"{p.get_height():.3f}",
                       (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha = 'center', va = 'center',
                       xytext = (0, 9),
                       textcoords = 'offset points')

plt.tight_layout()
plt.show()


BAGIAN 6: ANALISIS DAN VISUALISASI HASIL


NameError: name 'results' is not defined